# Tensorflow V2 API 활용하기
## (1) Sequential API

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train[...,np.newaxis]
x_test = x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [7]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32,3,activation='relu'))
model.add(keras.layers.Conv2D(64,3,activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

#model.summary()

In [9]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 17s 8ms/step - loss: 0.2113 - accuracy: 0.9341
Epoch 2/5
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0318 - accuracy: 0.9903
Epoch 3/5
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0181 - accuracy: 0.9942
Epoch 4/5
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0114 - accuracy: 0.9961
Epoch 5/5
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0081 - accuracy: 0.9972
313/313 - 2s - loss: 0.0444 - accuracy: 0.9887


[0.044361744076013565, 0.9886999726295471]

## (2) Functional API
* ```keras.Input(shape=(28,28,1)``` 먼저 선언
* ```output=predictions``` 출력단 설정

In [12]:
inputs = keras.Input(shape=(28,28,1))

x = keras.layers.Conv2D(32,3,activation='relu')(inputs)
x = keras.layers.Conv2D(64,3,activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [14]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train,y_train,epochs=5)

model.evaluate(x_test,y_test,verbose=2)

Epoch 1/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0073 - accuracy: 0.9978
Epoch 2/5
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0043 - accuracy: 0.9986
Epoch 3/5
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0048 - accuracy: 0.9984
Epoch 4/5
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0034 - accuracy: 0.9989
Epoch 5/5
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0055 - accuracy: 0.9984
313/313 - 1s - loss: 0.0754 - accuracy: 0.9865


[0.07540366053581238, 0.9865000247955322]

## (3) Subclassing 활용
* ```__init__()``` 메소드네안에서 레이어를 선언
* ```call()``` 메소드 안에서 forward propagation 구현

In [15]:
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(32,3,activation='relu')
        self.conv2 = keras.layers.Conv2D(64,3,activation='relu')
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(128,activation='relu')
        self.fc2 = keras.layers.Dense(10,activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
model = CustomModel()

## CIFAR-100 : (1) sequential

In [16]:
import tensorflow as tf
from tensorflow import keras

In [17]:
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0 , x_test / 255.0
print(len(x_train), len(x_test))

169009152/169001437 [==============================] - 2s 0us/step
50000 10000


In [24]:
model = keras.Sequential([
    keras.layers.Conv2D(16, 3, activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(100, activation='softmax')  
])

In [28]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5


InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [1152,100] and labels shape [32]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-28-2e6f44c4f44e>:5) ]] [Op:__inference_train_function_62570]

Function call stack:
train_function


## : (2) Functional

In [32]:
inputs = keras.Input(shape=(32,32,3))

x = keras.layers.Conv2D(16,3,activation='relu')(inputs)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Conv2D(32,3,activation='relu')(x)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256,activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [35]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train,y_train,epochs=5)
model.evaluate(x_test,y_test,verbose=2)

Epoch 1/5
1563/1563 [==============================] - 6s 3ms/step - loss: 2.6270 - accuracy: 0.3385
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.3979 - accuracy: 0.3877
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2184 - accuracy: 0.4260
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.0666 - accuracy: 0.4609
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 1.9431 - accuracy: 0.4819
313/313 - 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]

## : (3) Subclassing

In [38]:
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16,3,activation='relu')
        self.maxpool1 = keras.layers.MaxPooling2D((2,2))
        self.conv2 = keras.layers.Conv2D(32,3,activation='relu')
        self.maxpool2 = keras.layers.MaxPooling2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256,activation='relu')
        self.fc2 = keras.layers.Dense(100,activation='softmax')
        
    def call(self,x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
    
model = CustomModel()

In [39]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(x_train, y_train, epochs=1)
model.evaluate(x_test,y_test, verbose=2)

1563/1563 [==============================] - 6s 3ms/step - loss: 3.9912 - accuracy: 0.0969
313/313 - 1s - loss: 3.1549 - accuracy: 0.2329


[3.154888391494751, 0.2328999936580658]

## GradientTape 써보기
* 모델 학습 중 추상화된 설정 지점을 상세하게 지정
```tf.GradientTape```
1. Forward Propagation 수행 및 중간 레이어 값 저장
2. Loss 값 계산
3. 중간 레이어값 및 Loss를 활용한 체인룰(chain rule) 방식의 역전파 (Backward Propagation) 수행
4. 학습 파라미터 업데이트

In [41]:
import tensorflow as tf
from tensorflow import keras

cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16,3,activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32,3,activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
    
model = CustomModel()

50000 10000


* ```tape.gradient()```를 통해 매 스텝 학습이 진행될 때마다 발생하는 그래디언트를 추출한 후 
* ```optimizer.apply_gradients()```를 통해 발생한 그래디언트가 업데이트해야 할 파라미터 ```model.trainable_variables```를 지정해줌

In [46]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [47]:
import time
def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x,y) in enumerate(zip(x_train, y_train)):
            if step % batch_size == batch_size-1:
                x_batch.append(x)
                y_batch.append(y)
                loss = train_step(np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.float32))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))
        
train_model()

Epoch 0: last batch loss = 4.4405
Epoch 1: last batch loss = 4.3246
Epoch 2: last batch loss = 4.0950
Epoch 3: last batch loss = 4.4835
Epoch 4: last batch loss = 4.1087
It took 80.69533157348633 seconds


In [48]:
# evaluation
prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test) # accuracy

1/1 [==============================] - 4s 4s/step


0.0588